## Module 1 Homework

## Docker & SQL

In this homework we'll prepare the environment 
and practice with Docker and SQL

## Question 1. Knowing docker tags

Run the command to get information on Docker 

```docker --help```

Now run the command to get help on the "docker build" command:

```docker build --help```

Do the same for "docker run".

Which tag has the following text? - *Automatically remove the container when it exits* 

- `--delete`
- `--rc`
- `--rmc`
- `--rm`

### Answer :: `--rm`

## Question 2. Understanding docker first run 

Run docker with the python:3.9 image in an interactive mode and the entrypoint of bash.
Now check the python modules that are installed ( use ```pip list``` ). 

What is version of the package *wheel* ?

- 0.42.0
- 1.0.0
- 23.0.1
- 58.1.0

![](https://i.imgur.com/vTrrdF8.png)

### Answer :: `0.42.0`

# Prepare Postgres

Run Postgres and load data as shown in the videos
We'll use the green taxi trips from September 2019:

```wget https://github.com/DataTalksClub/nyc-tlc-data/releases/download/green/green_tripdata_2019-09.csv.gz```

**Answer => ::**

```shell
python \
    ./01-wk01-intro-plus-prerequi/src/01_pipeline_nyc_taxi_tripdata_to_localdb.py \
    --data-source "${PWD}/00-test-intermediate-artifacts/green_tripdata_2019-09.csv.gz" \
    --db-table-name green_tripdata_2019_09
```

![](https://i.imgur.com/EtfcUom.png)

![](https://i.imgur.com/fsLp6k6.png)

You will also need the dataset with zones:

```wget https://s3.amazonaws.com/nyc-tlc/misc/taxi+_zone_lookup.csv```

Download this data and put it into Postgres (with jupyter notebooks or with a pipeline)

**Answer => ::**

```shell
python \
    ./01-wk01-intro-plus-prerequi/src/02_pipeline_nyc_taxi_zone_lookup_to_localdb.py \
    --data-source "${PWD}/00-test-intermediate-artifacts/taxi+_zone_lookup.csv" \
    --db-table-name taxi_plus_zone_lookup
```

![](https://i.imgur.com/u0ncRdd.png)

![](https://i.imgur.com/Sr9mAKf.png)


Ensure you have `sqlalchemy` and `pandas` installed in your environment. If not, you can install them using pip:

In [1]:
%pip install sqlalchemy pandas python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
from sqlalchemy import create_engine
import pandas as pd
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Retrieve database URL and name from environment variables
db_url = os.getenv('DATABASE_URL')
db_name = os.getenv('DATABASE_NAME')
db_uri_path = f"{db_url}/{db_name}"

# Create a database connection
db_conn = create_engine(db_uri_path)

## Question 3. Count records 

How many taxi trips were totally made on September 18th 2019?

Tip: started and finished on 2019-09-18. 

Remember that `lpep_pickup_datetime` and `lpep_dropoff_datetime` columns are in the format timestamp (date and hour+min+sec) and not in date.

- 15767
- 15612
- 15859
- 89009

### **Answer :: =>** `15612`

```
SELECT COUNT(*) AS total_trips
FROM green_tripdata_2019_09
WHERE 
    lpep_pickup_datetime >= '2019-09-18 00:00:00' AND 
    lpep_pickup_datetime < '2019-09-19 00:00:00' AND 
    lpep_dropoff_datetime >= '2019-09-18 00:00:00' AND 
    lpep_dropoff_datetime < '2019-09-19 00:00:00';
```

![](https://i.imgur.com/yOdovwv.png)


In [3]:
# SQL query
sql_query = """
SELECT COUNT(*) AS total_trips
FROM green_tripdata_2019_09
WHERE 
    lpep_pickup_datetime >= '2019-09-18 00:00:00' AND 
    lpep_pickup_datetime < '2019-09-19 00:00:00' AND 
    lpep_dropoff_datetime >= '2019-09-18 00:00:00' AND 
    lpep_dropoff_datetime < '2019-09-19 00:00:00';
"""

# Execute the query and fetch the result into a DataFrame
result_df = pd.read_sql_query(sql_query, db_conn)

# Display the result
print(result_df)

   total_trips
0        15612


## Question 4. Largest trip for each day

Which was the pick up day with the largest trip distance
Use the pick up time for your calculations.

- 2019-09-18
- 2019-09-16
- 2019-09-26
- 2019-09-21

### **Answer ::** `2019-09-26`

In [4]:
# SQL query
sql_query = """
SELECT 
    DATE(lpep_pickup_datetime) AS pickup_day,
    MAX(trip_distance) AS max_trip_distance
FROM 
    green_tripdata_2019_09
GROUP BY 
    DATE(lpep_pickup_datetime)
ORDER BY max_trip_distance DESC
LIMIT 5;
"""

# Execute the query and fetch the result into a DataFrame
result_df = pd.read_sql_query(sql_query, db_conn)

# Display the result
print(result_df)

   pickup_day  max_trip_distance
0  2019-09-26             341.64
1  2019-09-21             135.53
2  2019-09-16             114.30
3  2019-09-28              89.64
4  2019-09-24              82.12


## Question 5. The number of passengers

Consider lpep_pickup_datetime in '2019-09-18' and ignoring Borough has Unknown

Which were the 3 pick up Boroughs that had a sum of total_amount superior to 50000?
 
- "Brooklyn" "Manhattan" "Queens"
- "Bronx" "Brooklyn" "Manhattan"
- "Bronx" "Manhattan" "Queens" 
- "Brooklyn" "Queens" "Staten Island"

### **Answer :: =>** `"Brooklyn" "Manhattan" "Queens"`

In [6]:
# SQL query
sql_query = """
SELECT 
    tpzl.borough,
    SUM(gt.total_amount) AS total_revenue
FROM 
    green_tripdata_2019_09 AS gt
JOIN 
    taxi_plus_zone_lookup AS tpzl
ON 
    gt.pulocationid = tpzl.locationid
WHERE 
    DATE(gt.lpep_pickup_datetime) = '2019-09-18'
    AND tpzl.borough <> 'Unknown'
GROUP BY 
    tpzl.borough
HAVING 
    SUM(gt.total_amount) > 50000
ORDER BY 
    total_revenue DESC
LIMIT 3;
"""

# Execute the query and fetch the result into a DataFrame
result_df = pd.read_sql_query(sql_query, db_conn)

# Display the result
print(result_df)

     borough  total_revenue
0   Brooklyn       96333.24
1  Manhattan       92271.30
2     Queens       78671.71


## Question 6. Largest tip

For the passengers picked up in September 2019 in the zone name Astoria which was the drop off zone that had the largest tip?
We want the name of the zone, not the id.

Note: it's not a typo, it's `tip` , not `trip`

- Central Park
- Jamaica
- JFK Airport
- Long Island City/Queens Plaza

### **Answer :: =>** `JFK Airport`

In [7]:
# SQL query
sql_query = """
SELECT 
    dropoff_zone.zone AS dropoff_zone_name,
    MAX(gtd.tip_amount) AS largest_tip
FROM 
    green_tripdata_2019_09 gtd
JOIN 
    taxi_plus_zone_lookup pickup_zone ON gtd.pulocationid = pickup_zone.locationid
JOIN 
    taxi_plus_zone_lookup dropoff_zone ON gtd.dolocationid = dropoff_zone.locationid
WHERE 
    pickup_zone.zone = 'Astoria'
    AND DATE(gtd.lpep_pickup_datetime) >= '2019-09-01' 
    AND DATE(gtd.lpep_pickup_datetime) < '2019-10-01'
GROUP BY 
    dropoff_zone_name
ORDER BY 
    largest_tip DESC
LIMIT 3;
"""

# Execute the query and fetch the result into a DataFrame
result_df = pd.read_sql_query(sql_query, db_conn)

# Display the result
print(result_df)

  dropoff_zone_name  largest_tip
0       JFK Airport        62.31
1          Woodside        30.00
2          Kips Bay        28.00


## Terraform

In this section homework we'll prepare the environment by creating resources in GCP with Terraform.

In your VM on GCP/Laptop/GitHub Codespace install Terraform. 
Copy the files from the course repo
[here](https://github.com/DataTalksClub/data-engineering-zoomcamp/tree/main/week_1_basics_n_setup/1_terraform_gcp/terraform) to your VM/Laptop/GitHub Codespace.

Modify the files as necessary to create a GCP Bucket and Big Query Dataset.


## Question 7. Creating Resources

After updating the main.tf and variable.tf files run:

```
terraform apply
```

Paste the output of this command into the homework submission form.

### **Answer :: =>**

> `terraform plan`

![](https://i.imgur.com/OczC07j.png)

> `terraform apply`

![](https://i.imgur.com/Yx8hTze.png)

> `terraform destroy`

![](https://i.imgur.com/l5rORgL.png)


```shell
terraform apply

Terraform used the selected providers to generate the following execution plan. Resource actions are indicated with the
following symbols:
  + create

Terraform will perform the following actions:

  # google_bigquery_dataset.demo_dataset will be created
  + resource "google_bigquery_dataset" "demo_dataset" {
      + creation_time              = (known after apply)
      + dataset_id                 = "shamb0_zcamp_2024_hcl_demo_v1_bq_dataset"
      + default_collation          = (known after apply)
      + delete_contents_on_destroy = false
      + effective_labels           = (known after apply)
      + etag                       = (known after apply)
      + id                         = (known after apply)
      + is_case_insensitive        = (known after apply)
      + last_modified_time         = (known after apply)
      + location                   = "US"
      + max_time_travel_hours      = (known after apply)
      + project                    = "shamb0-zoomcamp-lab-01"
      + self_link                  = (known after apply)
      + storage_billing_model      = (known after apply)
      + terraform_labels           = (known after apply)
    }

  # google_storage_bucket.demo-bucket will be created
  + resource "google_storage_bucket" "demo-bucket" {
      + effective_labels            = (known after apply)
      + force_destroy               = true
      + id                          = (known after apply)
      + location                    = "US"
      + name                        = "shamb0_zcamp_2024_hcl_demo_v1_bucket"
      + project                     = (known after apply)
      + public_access_prevention    = (known after apply)
      + self_link                   = (known after apply)
      + storage_class               = "STANDARD"
      + terraform_labels            = (known after apply)
      + uniform_bucket_level_access = (known after apply)
      + url                         = (known after apply)

      + lifecycle_rule {
          + action {
              + type = "AbortIncompleteMultipartUpload"
            }
          + condition {
              + age                   = 1
              + matches_prefix        = []
              + matches_storage_class = []
              + matches_suffix        = []
              + with_state            = (known after apply)
            }
        }
    }

Plan: 2 to add, 0 to change, 0 to destroy.

Do you want to perform these actions?
  Terraform will perform the actions described above.
  Only 'yes' will be accepted to approve.

  Enter a value: yes

google_bigquery_dataset.demo_dataset: Creating...
google_storage_bucket.demo-bucket: Creating...
google_bigquery_dataset.demo_dataset: Creation complete after 1s [id=projects/shamb0-zoomcamp-lab-01/datasets/shamb0_zcamp_2024_hcl_demo_v1_bq_dataset]
google_storage_bucket.demo-bucket: Creation complete after 2s [id=shamb0_zcamp_2024_hcl_demo_v1_bucket]

Apply complete! Resources: 2 added, 0 changed, 0 destroyed.
```